In [110]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

In [111]:
#Zastosowana normalizacja oparta jest na min-max. 
#Wartości są normalizowane na podstawie ich znaczenia i odchylen.
def normalizeZscore(v):
    return (v-np.mean(v,axis=0))/np.std(v,axis=0)

In [112]:
#Metoda przeprowadza liniową transformację pierwotnych danych do przedziału [0,1]
def normalizeMinMax(v):
    return (v-np.amin(v,axis=0))/(np.amax(v,axis=0)-np.amin(v,axis=0))

In [113]:
def splitIntoSection(normalizeDate,size = 500):
    # Funkcja odpowiedzialna za podzielenie sekcje według podanego segmentu
    # Muszę podzielić dane na segmenty. Próbe przeprowadzę dla segmentów 500 elementowych
    array = [np.zeros(size) for i in range(int(len(normalizeDate)/size))]
    i = 0
    n = 0
    while(i < len(array)):
        for j in range(len(array[i])):
            array[i][j] = normalizeDate[n]
            n+=1
        i+=1
    return array

In [114]:
def train_test_split(date,size):
    # Funkcja odpowiedzialna za podzielenie danych na dane treningowe i testowe 
    # Kolejnym etapem będzie podział na dane treningowe i testowe w stosunku 7:3. 
    size = round(len(date) * (size * 0.01))
    train = date[0:size]
    test = date[size:]
    return train,test

In [115]:
def openFile(file):
    # Funkcja odpowiedzialna za otwarcie i zmniejszenie częstotliwości próbkowania źródła
    # Część odpowiedzialna za zmniejszenie liczby sygnałów. Mamy zbyt częste próbkowanie (ok 100 MHz), dlatego 
    # muszę zmniejszyć częstotliwość próbkowania do ok 10 MHz.
    df = pd.read_csv(file, delimiter = "\t")
    dateToMuch = df.to_numpy()
    j = 0
    length = 0
    
    if len(dateToMuch) % 10 != 0:
        length = len(dateToMuch) - (len(dateToMuch) % 10)
    else:
        length = len(dateToMuch)
        
    date = np.zeros(int(len(dateToMuch)/10))
    for i in range(0,length,10):
        date[j] = dateToMuch[i]
        j+=1
    return date

In [116]:
def distances(train,x):
    return  np.sqrt(np.sum((x - train)**2, axis=1))

In [117]:
def k_nearest_classes(train, x, k=1):
    dist = distances(x)
    if k == 1:
        return train_labels[np.argmin(dist)]
    else:
        return train_labels[np.argsort(dist)[:k]]

In [118]:
def classify(x, k=1):
    if len(x.shape)>1:
        return np.array([np.bincount(k_nearest_classes(i,k)).argmax() for i in x ])
    return np.bincount(k_nearest_classes(x,k)).argmax()

In [121]:
def main(cell):
    
    # Otwarcie pliku i zmniejszenie próbkowania
    date = openFile('date/fibroblasty/f9.txt')
#     plt.figure(figsize=(16,4))
#     plt.grid(True)
#     plt.plot(date[0:],'b-')
    
    # Następnym elementem będzie normalizacja danych. Chcemy wykluczyć nagłe skoki sygnałowe.
    normalizeDate = normalizeZscore(date)
#     plt.figure(figsize=(16,4))
#     plt.grid(True)
#     plt.plot(normalizeDate[0:],'b-')
    
    # Podział na próbki 500 elementowe
    array = splitIntoSection(normalizeDate,500)
    
    # Podział na dane treningowe i testowe w stosunku 7:3
    traningX = [[] for i in range(len(array))]
    testX = [[] for i in range(len(array))]
    for i in range(len(array)):
        traningX[i],testX[i] =  train_test_split(array[i],70)
        
    # Utworzenie labely do tablic
    traningY = [[] for i in range(len(array))]
    testY = [[] for i in range(len(array))]
    for i in range(len(traningX)):
        for j in range(len(traningX[i])):
                traningY[i].append(cell)
    for i in range(len(testX)):
        for j in range(len(testX[i])):
            testY[i].append(cell)
#     print(len(traningY[0]))
#     print(len(testY[0]))

    # Wyćwiczenie klasyfikatora
    print(distances(traningX[0],testX[0][0]))
    
    
    

In [122]:
main('f')